*1. Нормализация данных*

In [1]:
import json
import pandas as pd

In [2]:
with open('response_example.json', 'r+', encoding='utf-8') as f:
    data = json.loads(f.read())

In [3]:
some_list = []
for i in data["data"]:
    inner_data = i.get("data")
    new_data = {
        "ID Gov": inner_data.get("id"),
        "Ссылка": [_.get("url") for _ in inner_data.get("images", [])],
        "Название": inner_data.get("name"),
        "Место отправления": inner_data.get("productionPlace"),
        "Место получения": inner_data.get("findPlace"),
        "Описание": inner_data.get("description"),
        "Дата": inner_data.get("periodStr"),
        "Вид бланка": inner_data.get("technologies", []),
    }    
    some_list.append(new_data)

In [4]:
telegrams = pd.DataFrame(some_list)
telegrams.to_excel("./telegrams.xlsx")

In [5]:
df = pd.read_excel('./telegrams.xlsx')
df = df.drop(df.columns[0], axis=1)

In [6]:
# присваиваем телеграммам ID

id = []
for i in range(55,1055):
    id.append("tg" + str("%05d" % (i,)))
df.insert(0, "ID", id)

# удаляем лишние "телеграммы" по айдишкам

id_to_drop = ['tg00302', 'tg00352', 'tg00355', 'tg00463', 'tg00484', 'tg00577', 'tg00763', 'tg00766', 'tg00837', 'tg00943', 'tg00968']
for index, row in df.iterrows():
    if row["ID"] in id_to_drop:
        df = df.drop(index)

# переприсваиваем ID без удалённых телеграмм

new_id = []
for i in range(55,1044):
    new_id.append("tg" + str("%05d" % (i,)))
df = df.drop(df.columns[0], axis=1)
df.insert(0, "ID", new_id)

In [7]:
# заполняем все пустые строки NA, убираем лишние символы

df = df.fillna("NA")
df = df.replace("\[\]", "NA", regex=True)
df = df.replace("[\['\]]", "", regex=True)

In [8]:
# вставляем поле "текст"

df.insert(2, "Текст", "")

for index, row in df.iterrows():
    if row["Ссылка"] == "NA":
        row["Текст"] = "NA"

In [9]:
df.to_excel("./new_telegrams.xlsx")

*2. Named Entity Recognition*

In [5]:
import re
import spacy
import pymorphy2

In [6]:
morph = pymorphy2.MorphAnalyzer(lang='ru')
nlp = spacy.load("ru_core_news_md")

In [ ]:
# вставляем колонки "отправитель" и "получатель"

df.insert(4, "Отправитель", "")
df.insert(5, "Получатель", "")

In [ ]:
for index, row in df.iterrows():
    # убираем лишние слэши и скобки (в них были инициалы, что мешало)
    sub_1 = re.sub(r"[\/\[\]]", r"", row["Название"])
    # убираем пробелы между инициалами, так лучше работает NER
    sub_2 = re.sub(r"([А-Я]{1}\.)(\s)([А-Я]{1}\.)", r"\1\3", sub_1)
    # чистим двойные пробелы
    sub_3 = re.sub(r"(\s)(\s)", r"\1", sub_2)
    # убираем слово "телеграмма" и возможную пунктуацию после него (опять же, так лучше парсится)
    sub_4 = re.sub(r"т?Телеграмма\.?,?\s?", r"", sub_3)
    doc = nlp(sub_4)
    for ent in doc.ents:
        p = morph.parse(ent.text)[0]
        # ищем персоналии в именительном падеже, лемматизируем и делаем их с большой буквы после лемматизации
        if ent.label_ == "PER" and p.tag.case == "nomn":
            row["Отправитель"] = morph.parse(ent.text)[0].normal_form.title()
        # ищем персоналии в родительном падеже
        elif ent.label_ == "PER" and p.tag.case == "gent":
            row["Отправитель"] = morph.parse(ent.text)[0].normal_form.title()
        # ищем организации в родительном падеже
        elif ent.label_ == "ORG" and p.tag.case == "gent":
            row["Отправитель"] = morph.parse(ent.text)[0].normal_form.title()
        # ищем персоналии в дательном падеже
        elif ent.label_ == "PER" and p.tag.case == "datv":
            row["Получатель"] = morph.parse(ent.text)[0].normal_form.title()
        # ищем организации в дательном падеже
        elif ent.label_ == "ORG" and p.tag.case == "datv":
            row["Получатель"] = morph.parse(ent.text)[0].normal_form.title()

    # меняем местами фамилию и инициалы

    row["Отправитель"] = re.sub(r"([А-ЯЁ]{1}\.[А-ЯЁ]{1}\.)(\s)([А-я]+)", r"\3\2\1", row["Отправитель"])
    row["Получатель"] = re.sub(r"([А-ЯЁ]{1}\.[А-ЯЁ]{1}\.)(\s)([А-я]+)", r"\3\2\1", row["Получатель"])

# заполняем все пустые ячейки NA

df = df.fillna("NA")

In [ ]:
df.to_excel("./new_telegrams.xlsx")

*3. Проставление других параметров в таблицу*

In [ ]:
df.head()

In [ ]:
df.insert(11, "Вид телеграммы", "")
df.insert(12, "Тема", "")

In [ ]:
for index, row in df.iterrows():
    row["Название"].casefold()
    row["Вид бланка"].casefold()

    if "присвоени" in row["Название"]:
        row["Тема"] = "присвоение звания"
    elif "приглашение" in row["Название"]:
        row["Тема"] = "приглашение"
    elif "новогод" or "новым годом" in row["Название"]:
        row["Тема"] = "новый год"

    if "правительственная" in row["Название"]:
        row["Вид телеграммы"] = "правительственная"
    elif "транзитная" in row["Название"]:
        row["Вид телеграммы"] = "транзитная"
    elif "срочная" in row["Название"]:
        row["Вид телеграммы"] = "срочная"
    elif "поздравительная" in row["Название"]:
        row["Вид телеграммы"] = "поздравительная"
    else:
        row["Вид телеграммы"] = "обычная"

    if "машинопись" in row["Вид бланка"]:
        row["Вид бланка"] = "печатный"
    elif "телеграфный бланк" in row["Вид бланка"]:
        row["Вид бланка"] = "печатный"
    elif "бумага газетная" in row["Вид бланка"]:
        row["Вид бланка"] = "печатный"
    elif "фотобумага" in row["Вид бланка"]:
        row["Вид бланка"] = "открытка"
    elif "рукопись" in row["Вид бланка"]:
        row["Вид бланка"] = "рукопись"


In [ ]:
df.to_excel("./new_telegrams.xlsx")